# Neural Network Deep Dive

We will first demonstrate how neural networks work using the Numpy module.

In [1]:
import numpy as np

In [2]:
W1 = np.random.randn(n_h,n_x) * 0.01
b1 = np.zeros(shape=(n_h, 1))
W2 = np.random.randn(n_y,n_h) * 0.01
b2 = np.zeros(shape=(n_y, 1))

NameError: name 'n_h' is not defined